In [25]:
#Abstractive Summarization with Hugging Face Transformers
#Uses a pre-trained model like BART
#Use this for natural language summaries (more human-like)

import pandas as pd
import time
from transformers import pipeline

# Load the summarization model
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")

# Set your CSV file path here
csv_path = r"C:\Users\salsubhi1\PycharmProjects\Enigmatic Research\excel_chunks_text.csv"  # Change this to your actual file path

# Read CSV file
df = pd.read_csv(csv_path)

# Assuming the text to summarize is in a column named "text"
text_to_summarize = " ".join(df["text"].dropna().astype(str).tolist())

# Measure time taken for summarization
start_time = time.time()

# Make sure the text isn't too short or too long for the model
if len(text_to_summarize) > 1024:
    text_to_summarize = text_to_summarize[:1024]  # You can chunk it later for full summarization

# Generate summary
summary = summarizer(text_to_summarize, max_length=150, min_length=50, do_sample=False)

# End time measurement
end_time = time.time()

# Calculate time taken and tokens per second
duration = end_time - start_time
tokens_processed = len(text_to_summarize.split())  # Count the number of tokens in the input text
tokens_per_second = tokens_processed / duration if duration > 0 else 0

# Print the summary and performance metrics
print("BART Summary:\n", summary[0]['summary_text'])
print(f"Tokens processed: {tokens_processed}")
print(f"Time taken: {duration:.4f} seconds")
print(f"Tokens per second: {tokens_per_second:.2f}")


Device set to use cpu


Summary:
 Loan Amortisation Schedule: Sheet: Loan Schedule                 Unnamed: 0       unnamed: 1   U.N. Loan amount: $5,000; $10,000 for non-concessional loans; $15,000 on a secured line of credit; $20,000 if the line is unsecured; $25,000 with a secured loan.
Tokens processed: 71
Time taken: 7.1216 seconds
Tokens per second: 9.97


In [27]:
#T5 Model with Transformers
#t5-small handles up to 512 tokens — not characters — so we split the text by word count.
import pandas as pd
import time
from transformers import T5Tokenizer, T5ForConditionalGeneration

# -------- Step 1: Extract text from CSV --------
def extract_csv_text(path, column_name="text"):
    df = pd.read_csv(path)
    return " ".join(df[column_name].dropna().astype(str).tolist())

# -------- Step 2: Chunk text into 512-token pieces --------
def chunk_text(text, chunk_size=500):
    words = text.split()
    return [" ".join(words[i:i+chunk_size]) for i in range(0, len(words), chunk_size)]

# -------- Step 3: Load T5 model and tokenizer --------
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")

# -------- Step 4: Summarize each chunk --------
def summarize_with_t5(text_chunks):
    summaries = []
    for chunk in text_chunks:
        input_text = "summarize: " + chunk
        input_ids = tokenizer.encode(input_text, return_tensors="pt", max_length=512, truncation=True)

        output_ids = model.generate(
            input_ids,
            max_length=150,
            min_length=40,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        summary = tokenizer.decode(output_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    return " ".join(summaries)

# -------- Step 5: Performance Measurement --------
def measure_performance(csv_path, column_name="text"):
    # Extract text
    full_text = extract_csv_text(csv_path, column_name)
    
    # Measure time taken for chunking and summarization
    start_time = time.time()
    
    # Chunk the text
    text_chunks = chunk_text(full_text)
    
    # Summarize text
    final_summary = summarize_with_t5(text_chunks)
    
    # End time measurement
    end_time = time.time()

    # Calculate performance metrics
    duration = end_time - start_time
    tokens_processed = len(full_text.split())  # Count the number of tokens in the input text
    tokens_per_second = tokens_processed / duration if duration > 0 else 0
    
    # Print the results
    print("T5 Summary:\n", final_summary)
    print(f"\nTokens processed: {tokens_processed}")
    print(f"Time taken: {duration:.4f} seconds")
    print(f"Tokens per second: {tokens_per_second:.2f}")

# Set your CSV file path here
csv_path = r"C:\Users\salsubhi1\PycharmProjects\Enigmatic Research\excel_chunks_text.csv"

# Run the performance measurement and summarization
measure_performance(csv_path)



T5 Summary:
 0 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN Na 82.778669 69.058164 55.459760 47.421131 42.160403 NaN NaN NaN NaN 0.0325 291.915152 180.800023 97.719029 83.956912 70.266877 56.719576 48.731623 43.520632 NaN NaN 0.035 293.020797 181.91745 98.885867 85.145374 71.488254 57.995972 50 NaN NaN 0.135 339.352873 230.09846 152.274289 140.571716 129.831854 120.737468 116.564488 114.541218 NaN NaN 0.1375 340.563328 231.388449 153.766812 142.138275 131.498727 122.540538 118.466597 116.511253 NaN NaN NaN 0.14 341.776 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN Na NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN Na

In [28]:
#TextRank Summarization with Sumy
import pandas as pd
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.text_rank import TextRankSummarizer
import time

# -------- Step 1: Extract text from CSV --------
def extract_csv_text(path, column_name="text"):
    df = pd.read_csv(path)
    return " ".join(df[column_name].dropna().astype(str).tolist())

# -------- Step 2: Summarize the extracted text --------
def summarize_csv_text(csv_path, column_name="text"):
    # Extract the text from the CSV
    text = extract_csv_text(csv_path, column_name)

    # Measure the performance (start time)
    start_time = time.time()
    
    # Use the TextRank Summarizer
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = TextRankSummarizer()
    summary = summarizer(parser.document, sentences_count=5)  # Number of sentences in the summary

    # End time measurement
    end_time = time.time()

    # Calculate performance metrics
    duration = end_time - start_time
    tokens_processed = len(text.split())  # Count the number of tokens in the input text
    tokens_per_second = tokens_processed / duration if duration > 0 else 0

    # Output the summary
    print("TextRank Summary:\n")
    for sentence in summary:
        print(sentence)

    # Print performance metrics
    print(f"\nTokens processed: {tokens_processed}")
    print(f"Time taken: {duration:.4f} seconds")
    print(f"Tokens per second: {tokens_per_second:.2f}")

# -------- Step 3: Run the summarization --------
csv_path = r"C:\Users\salsubhi1\PycharmProjects\Enigmatic Research\excel_chunks_text.csv"
summarize_csv_text(csv_path, column_name="text")  # Replace "text" if the column has a different name


TextRank Summary:

Sheet: Loan Schedule Unnamed: 0                  Unnamed: 1                 Unnamed: 2         Unnamed: 3          Unnamed: 4     Unnamed: 5                   Unnamed: 6  Unnamed: 7     Unnamed: 8      Unnamed: 9          Unnamed: 10 NaN                         NaN Loan Amortisation Schedule                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN NaN                         NaN                        NaN                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN NaN                Enter Values                        NaN                NaN                 NaN            NaN                 Loan Summary         NaN            NaN             NaN                  NaN NaN                 Loan amount                        NaN                NaN                5000            NaN            Sched

In [31]:
#Luhn Summarizer (Sumy)
import pandas as pd
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.luhn import LuhnSummarizer
import time

# -------- Step 1: Extract text from CSV --------
def extract_csv_text(path, column_name="text"):
    df = pd.read_csv(path)
    return " ".join(df[column_name].dropna().astype(str).tolist())

# -------- Step 2: Summarize the extracted text --------
def summarize_csv_text(csv_path, column_name="text"):
    # Extract text
    text = extract_csv_text(csv_path, column_name)

    # Start performance timer
    start_time = time.time()

    # Luhn summarization
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LuhnSummarizer()
    summary = summarizer(parser.document, sentences_count=5)

    # End performance timer
    end_time = time.time()

    # Performance metrics
    duration = end_time - start_time
    tokens_processed = len(text.split())
    tokens_per_second = tokens_processed / duration if duration > 0 else 0

    # Output summary
    print("Luhn Summary:\n")
    for sentence in summary:
        print(sentence)

    # Output performance
    print(f"\nTokens processed: {tokens_processed}")
    print(f"Time taken: {duration:.4f} seconds")
    print(f"Tokens per second: {tokens_per_second:.2f}")

# -------- Step 3: Run the summarization --------
csv_path = r"C:\Users\salsubhi1\PycharmProjects\Enigmatic Research\excel_chunks_text.csv"
summarize_csv_text(csv_path, column_name="text")



Luhn Summary:

Sheet: Loan Schedule Unnamed: 0                  Unnamed: 1                 Unnamed: 2         Unnamed: 3          Unnamed: 4     Unnamed: 5                   Unnamed: 6  Unnamed: 7     Unnamed: 8      Unnamed: 9          Unnamed: 10 NaN                         NaN Loan Amortisation Schedule                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN NaN                         NaN                        NaN                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN NaN                Enter Values                        NaN                NaN                 NaN            NaN                 Loan Summary         NaN            NaN             NaN                  NaN NaN                 Loan amount                        NaN                NaN                5000            NaN            Scheduled

In [33]:
#SpaCy + Sentence Scoring (Custom Extractive)
import pandas as pd
import spacy
import time

# -------- Step 1: Extract text from CSV --------
def extract_csv_text(path, column_name="text"):
    df = pd.read_csv(path)
    return " ".join(df[column_name].dropna().astype(str).tolist())

# -------- Step 2: Summarize the extracted text using SpaCy --------
def summarize_csv_text(csv_path, column_name="text"):
    # Load the SpaCy model
    nlp = spacy.load("en_core_web_sm")

    # Extract text from the CSV file
    text = extract_csv_text(csv_path, column_name)

    # Start performance timer
    start_time = time.time()

    # Process the text with SpaCy
    doc = nlp(text)

    # Dictionary to store sentence scores
    sentence_scores = {}

    # Simple heuristic: score = number of named entities + noun chunks
    for sent in doc.sents:
        score = len(list(sent.ents)) + len(list(sent.noun_chunks))
        sentence_scores[sent] = score

    # Get top 5 sentences based on the highest scores
    top_sentences = sorted(sentence_scores, key=sentence_scores.get, reverse=True)[:5]

    # End performance timer
    end_time = time.time()

    # Calculate performance metrics
    duration = end_time - start_time
    tokens_processed = len(text.split())
    tokens_per_second = tokens_processed / duration if duration > 0 else 0

    # Output the summary
    print("SpaCy Custom Extractive Summary:\n")
    for sentence in top_sentences:
        print(sentence.text.strip())

    # Output performance metrics
    print(f"\nTokens processed: {tokens_processed}")
    print(f"Time taken: {duration:.4f} seconds")
    print(f"Tokens per second: {tokens_per_second:.2f}")

# -------- Step 3: Run the summarization --------
csv_path = r"C:\Users\salsubhi1\PycharmProjects\Enigmatic Research\excel_chunks_text.csv"
summarize_csv_text(csv_path, column_name="text")  # Change "text" if your column has a different name


SpaCy Custom Extractive Summary:

Unnamed: 15   Unnamed: 16 Unnamed: 17
        NaN               PARTY BUDGET        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                    SUMMARY  ESTIMATED     ACTUAL OVER/UNDER         NaN      SUMMARY OF ACTUAL EXPENSES        NaN        NaN        NaN   

In [34]:
#Summarize Entire CSV Text Column with PEGASUS
import pandas as pd
import time
from transformers import PegasusTokenizer, PegasusForConditionalGeneration

# -------- Step 1: Load CSV and extract 'text' column --------
def load_csv_text(csv_path, text_column="text"):
    df = pd.read_csv(csv_path)
    text_data = df[text_column].dropna().astype(str).tolist()
    return " ".join(text_data)  # Combine all rows into one document

# -------- Step 2: Chunk text to fit Pegasus input size (1024 tokens) --------
def chunk_text(text, tokenizer, max_tokens=1024):
    words = text.split()
    chunks = []
    current_chunk = []

    for word in words:
        current_chunk.append(word)
        if len(tokenizer.encode(" ".join(current_chunk))) > max_tokens:
            current_chunk.pop()  # Remove the last word that pushed it over
            chunks.append(" ".join(current_chunk))
            current_chunk = [word]

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

# -------- Step 3: Generate summary using PEGASUS --------
def summarize_text(text_chunks, model, tokenizer):
    summaries = []
    for chunk in text_chunks:
        inputs = tokenizer(chunk, truncation=True, padding="longest", return_tensors="pt")
        summary_ids = model.generate(
            **inputs,
            max_length=100,
            min_length=30,
            length_penalty=2.0,
            num_beams=4,
            early_stopping=True
        )
        summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
        summaries.append(summary)
    return " ".join(summaries)

# -------- Step 4: Main function --------
def summarize_csv(csv_path, text_column="text"):
    print("Loading PEGASUS model...")
    tokenizer = PegasusTokenizer.from_pretrained("google/pegasus-xsum")
    model = PegasusForConditionalGeneration.from_pretrained("google/pegasus-xsum")

    print("Reading CSV and preparing text...")
    full_text = load_csv_text(csv_path, text_column)

    # Start timing
    start_time = time.time()

    text_chunks = chunk_text(full_text, tokenizer)

    print(f"Text divided into {len(text_chunks)} chunk(s). Generating summary...")
    final_summary = summarize_text(text_chunks, model, tokenizer)

    # End timing
    end_time = time.time()
    duration = end_time - start_time
    token_count = len(full_text.split())
    tokens_per_sec = token_count / duration if duration > 0 else 0

    # Output summary
    print("\n✨ PEGASUS Summary:\n")
    print(final_summary)

    # Output performance metrics
    print(f"\nTokens processed: {token_count}")
    print(f"Time taken: {duration:.2f} seconds")
    print(f"Tokens per second: {tokens_per_sec:.2f}")

# -------- Run it --------
if __name__ == "__main__":
    csv_file_path = r"C:\Users\salsubhi1\PycharmProjects\Enigmatic Research\excel_chunks_text.csv"  # Update your path
    summarize_csv(csv_file_path, text_column="text")


Loading PEGASUS model...


Some weights of PegasusForConditionalGeneration were not initialized from the model checkpoint at google/pegasus-xsum and are newly initialized: ['model.decoder.embed_positions.weight', 'model.encoder.embed_positions.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Token indices sequence length is longer than the specified maximum sequence length for this model (515 > 512). Running this sequence through the model will result in indexing errors


Reading CSV and preparing text...
Text divided into 6 chunk(s). Generating summary...

✨ PEGASUS Summary:

The following table lists the outstanding loans of the Republic of Ireland at the end of the first quarter of the current financial year, as reported by the Irish Central Bank (ICB). Match reports and team news for Saturday's Premier League game between Arsenal and Tottenham at Emirates Stadium in London, as Arsene Wenger's side take on Mauricio Pochettino's side. Here is a list of the key raw materials used in the production of NaN NaN, as reported by the US Geological Survey (USGS) in the third quarter of 2016. All photographs courtesy of AFP, EPA, Getty Images, Reuters, Getty Images and Reuters news agency.''Source: AFP, EPA, Getty Images and Reuters news agency Find out more about the events at this year's White House Correspondents' Association dinner at the Washington Hilton Hotel in Washington, DC, on Saturday, April 16. Here is a breakdown of some of the costs involved in 

In [19]:
#Comparing PEGASUS with BART  
from rouge_score import rouge_scorer

# Define your summaries here
reference_summary = """The following table lists the outstanding loans of the Republic of Ireland at the end of the first quarter of the current financial year, as reported by the Irish Central Bank (ICB). Match reports and team news for Saturday's Premier League game between Arsenal and Tottenham at Emirates Stadium in London, as Arsene Wenger's side take on Mauricio Pochettino's side. Here is a list of the key raw materials used in the production of NaN NaN, as reported by the US Geological Survey (USGS) in the third quarter of 2016. All photographs courtesy of AFP, EPA, Getty Images, Reuters, Getty Images and Reuters news agency.''Source: AFP, EPA, Getty Images and Reuters news agency Find out more about the events at this year's White House Correspondents' Association dinner at the Washington Hilton Hotel in Washington, DC, on Saturday, April 16. Here is a breakdown of some of the costs involved in getting married in Dublin, as compiled by the city's Department of Tourism and Sport."""
generated_summary = """Loan Amortisation Schedule: Sheet: Loan Schedule                 Unnamed: 0       unnamed: 1   U.N. Loan amount: $5,000; $10,000 for non-concessional loans; $15,000 on a secured line of credit; $20,000 if the line is unsecured; $25,000 with a secured loan."""

# Create a ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Score
scores = scorer.score(reference_summary, generated_summary)

# Print results
print("ROUGE Scores PEGASUS with BART:")
for metric, score in scores.items():
    print(f"{metric}: Precision: {score.precision:.4f}, Recall: {score.recall:.4f}, F1: {score.fmeasure:.4f}")


ROUGE Scores PEGASUS with BART:
rouge1: Precision: 0.1860, Recall: 0.0471, F1: 0.0751
rouge2: Precision: 0.0000, Recall: 0.0000, F1: 0.0000
rougeL: Precision: 0.1860, Recall: 0.0471, F1: 0.0751


In [20]:
#Comparing PEGASUS with T5
from rouge_score import rouge_scorer

# Define your summaries here
reference_summary = """The following table lists the outstanding loans of the Republic of Ireland at the end of the first quarter of the current financial year, as reported by the Irish Central Bank (ICB). Match reports and team news for Saturday's Premier League game between Arsenal and Tottenham at Emirates Stadium in London, as Arsene Wenger's side take on Mauricio Pochettino's side. Here is a list of the key raw materials used in the production of NaN NaN, as reported by the US Geological Survey (USGS) in the third quarter of 2016. All photographs courtesy of AFP, EPA, Getty Images, Reuters, Getty Images and Reuters news agency.''Source: AFP, EPA, Getty Images and Reuters news agency Find out more about the events at this year's White House Correspondents' Association dinner at the Washington Hilton Hotel in Washington, DC, on Saturday, April 16. Here is a breakdown of some of the costs involved in getting married in Dublin, as compiled by the city's Department of Tourism and Sport."""
generated_summary = """0 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4 Unnamed: 5 Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9 Unnamed: 10 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN Na 82.778669 69.058164 55.459760 47.421131 42.160403 NaN NaN NaN NaN 0.0325 291.915152 180.800023 97.719029 83.956912 70.266877 56.719576 48.731623 43.520632 NaN NaN 0.035 293.020797 181.91745 98.885867 85.145374 71.488254 57.995972 50 NaN NaN 0.135 339.352873 230.09846 152.274289 140.571716 129.831854 120.737468 116.564488 114.541218 NaN NaN 0.1375 340.563328 231.388449 153.766812 142.138275 131.498727 122.540538 118.466597 116.511253 NaN NaN NaN 0.14 341.776 NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN Na NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN NaN Na """

# Create a ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Score
scores = scorer.score(reference_summary, generated_summary)

# Print results
print("ROUGE Scores PEGASUS with T5:")
for metric, score in scores.items():
    print(f"{metric}: Precision: {score.precision:.4f}, Recall: {score.recall:.4f}, F1: {score.fmeasure:.4f}")


ROUGE Scores PEGASUS with T5:
rouge1: Precision: 0.0076, Recall: 0.0118, F1: 0.0093
rouge2: Precision: 0.0038, Recall: 0.0059, F1: 0.0047
rougeL: Precision: 0.0076, Recall: 0.0118, F1: 0.0093


In [21]:
#Comparing PEGASUS with TextRank   
from rouge_score import rouge_scorer

# Define your summaries here
reference_summary = """The following table lists the outstanding loans of the Republic of Ireland at the end of the first quarter of the current financial year, as reported by the Irish Central Bank (ICB). Match reports and team news for Saturday's Premier League game between Arsenal and Tottenham at Emirates Stadium in London, as Arsene Wenger's side take on Mauricio Pochettino's side. Here is a list of the key raw materials used in the production of NaN NaN, as reported by the US Geological Survey (USGS) in the third quarter of 2016. All photographs courtesy of AFP, EPA, Getty Images, Reuters, Getty Images and Reuters news agency.''Source: AFP, EPA, Getty Images and Reuters news agency Find out more about the events at this year's White House Correspondents' Association dinner at the Washington Hilton Hotel in Washington, DC, on Saturday, April 16. Here is a breakdown of some of the costs involved in getting married in Dublin, as compiled by the city's Department of Tourism and Sport."""
generated_summary = """Sheet: Loan Schedule Unnamed: 0                  Unnamed: 1                 Unnamed: 2         Unnamed: 3          Unnamed: 4     Unnamed: 5                   Unnamed: 6  Unnamed: 7     Unnamed: 8      Unnamed: 9          Unnamed: 10 NaN                         NaN Loan Amortisation Schedule                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN NaN                         NaN                        NaN                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN NaN                Enter Values                        NaN                NaN                 NaN            NaN                 Loan Summary         NaN            NaN             NaN                  NaN NaN                 Loan amount                        NaN                NaN                5000            NaN            Scheduled payment         NaN     425.749521             NaN                  NaN NaN        Annual interest rate                        NaN                NaN                0.04            NaN Scheduled number of payments         NaN             12             NaN                  NaN NaN        Loan period in years                        NaN                NaN                   1            NaN    Actual number of payments         NaN             10             NaN                  NaN NaN Number of payments per year                        NaN                NaN                  12            NaN         Total early payments         NaN            900             NaN                  NaN NaN          Start date of loan                        NaN                NaN 2025-04-07 00:00:00            NaN               Total interest         NaN      89.621486             NaN                  NaN NaN                         NaN                        NaN                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN NaN     Optional extra payments                        NaN                NaN                 100            NaN                  Lender name         NaN Woodgrove Bank             NaN                  NaN NaN                         NaN                        NaN                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN NaN              Payment Number              Payment\nDate Beginning\nBalance   Scheduled Payment Extra\nPayment               Total\nPayment   Principal       Interest Ending\nBalance Cumulative\nInterest NaN                           1        2025-04-07 00:00:00               5000          425.749521            100                   525.749521  509.082854      16.666667     4490.917146            16.666667 NaN                           2        2025-05-07 00: 00:00        4490.917146          425.749521            100                   525.749521  510.779797      14.969724     3980.137349             31.63639 NaN                           3        2025-06-07 00:00:00        3980.137349          425.749521            100                   525.749521  512.482396      13.267124     3467.654952            44.903515 NaN                           4        2025-07-07 00:00:00        3467.654952          425.749521            100                   525.749521  514.190671       11.55885     2953.464281            56.462365 NaN                           5        2025-08-07 00:00:00        2953.464281          425.749521            100                   525.749521   515.90464       9.844881     2437.559641            66.307246 NaN                           6        2025-09-07 00:00:00        2437.559641          425.749521            100                   525.749521  517.624322       8.125199     1919.935319            74.432445 NaN                           7        2025-10-07 00:00:00        1919.935319          425.749521            100                   525.749521  519.349737       6.399784     1400.585582            80.832229 NaN                           8        2025-11-07 00:00:00        1400.585582          425.749521            100                   525.749521  521.080902       4.668619       879.50468            85.500848 NaN                           9        2025-12-07 00:00:00          879.50468          425.749521            100                   525.749521  522.817839        2.931682      356.686841             88.43253 NaN                          10        2026-01-07 00:00:00         356.686841          425.749521              0                   356.686841  355.497885       1.188956               0            89.621486 Sheet: LOAN ANALYSIS Unnamed: 0 LOAN ANALYSIS WORKSHEET  Unnamed: 2    Unnamed: 3  Unnamed: 4      Unnamed: 5  Unnamed: 6  Unnamed: 7   Unnamed: 8  Unnamed: 9  Unnamed: 10                                                      Unnamed: 11 NaN           LOAN ANALYSIS         NaN           NaN         NaN             NaN         NaN         NaN          NaN         NaN          NaN Rate slicer to filter table data based on Rates is in this cell.
NaN           INTEREST RATE         NaN          0.05         NaN MONTHLY PAYMENT         NaN         NaN   106.065515         NaN          NaN                                                              NaN NaN           YEARS OF LOAN         NaN            10         NaN   TOTAL PAYMENT         NaN         NaN 12727.861829         NaN          NaN                                                              NaN NaN             LOAN AMOUNT         NaN         10000         NaN  TOTAL INTEREST         NaN         NaN  2727.861829         NaN          NaN                                                              NaN NaN            PAYMENTS DUE         NaN End of Period         NaN             NaN         NaN         NaN          NaN         NaN          NaN                                                              NaN NaN                     NaN       YEARS           NaN         NaN             NaN         NaN         NaN          NaN         NaN          NaN                                                              NaN NaN                    RATE           3             5   10.000000              12   15.000000   20.000000    25.000000   30.000000          NaN                                                              NaN NaN                    0.02  286.425787    175.277601   92.013454       78.168369   64.350870   50.588334    42.385434   36.961947          NaN                                                              NaN NaN                  0.0225  287.518475    176.373448   93.137372       79.305498   65.508477   51.780829    43.613070   38.224610          NaN                                                              NaN NaN                   0.025  288.613757    177.473616   94.269902       80.452938   66.678921   52.990289    44.861673    39.512090          NaN                                                              NaN NaN                  0.0275  289.711631    178.578103   95.411031        81.61067   67.862164   54.216631    46.131086   40.824118          NaN                                                              NaN NaN                    0.03  290.812096    179.686907   96.560745       82.778669   69.058164   55.459760    47.421131   42.160403          NaN                                                              NaN NaN                  0.0325  291.915152    180.800023   97.719029       83.956912   70.266877   56.719576    48.731623   43.520632          NaN                                                              NaN NaN                   0.035  293.020797     181.91745   98.885867       85.145374   71.488254   57.995972    50.062357   44.904469          NaN                                                              NaN NaN                  0.0375   294.12903    183.039183  100.061243       86.344025   72.722244   59.288831    51.413120   46.311559          NaN                                                              NaN NaN                    0.04   295.23985    184.165221  101.245138       87.552837   73.968793   60.598033    52.783684   47.741530          NaN                                                              NaN NaN                  0.0425  296.353255    185.295558  102.437533       88.771778   75.227841   61.923447    54.173810   49.193989          NaN                                                              NaN NaN                   0.045  297.469245    186.430192  103.638409       90.000816   76.499329   63.264938    55.583248   50.668531          NaN                                                              NaN NaN                  0 .0475  298.587817     187.56912  104.847743       91.239916   77.783192   64.622363    57.011736   52.164734          NaN                                                              NaN NaN                    0.05  299.708971    188.712336  106.065515       92.489041   79.079363   65.995574    58.459004   53.682162          NaN                                                              NaN NaN                  0.0525  300.832705    189.859838  107.291701       93.748155   80.387772   67.384417    59.924772   55.220370          NaN                                                              NaN NaN                   0.055  301.959018    191.011622  108.526278       95.017217   81.708345   68.788731    61.408749   56.778900          NaN                                                              NaN NaN                  0.0575  303.087908    192.167682  109.769220       96.296187   83.041009   70.208351    62.910640   58.357286          NaN                                                              NaN NaN                    0.06  304.219375    193.328015  111.020502       97.585021   84.385683   71.643106    64.430140   59.955053          NaN                                                              NaN NaN                  0.0625  305.353415    194.492617  112.280097       98.883677   85.742287   73.092820    65.966938   61.571720          NaN                                                              NaN NaN                   0.065  306.490029    195.661482  113.547977      100.192109   87.110737   74.557314    67.520716   63.206802          NaN                                                              NaN NaN                  0.0675  307.629214    196.834607   114.824114      101.510269   88.490946   76.036401    69.091153   64.859810          NaN                                                              NaN NaN                    0.07  308.770969    198.011985  116.108479       102.83811   89.882827   77.529894    70.677920   66.530250          NaN                                                              NaN NaN                  0.0725  309.915292    199.193614  117.401041      104.175581   91.286288   79.037598    72.280686   68.217628          NaN                                                              NaN NaN                   0.075  311.062182    200.379486  118.701769      105.522631   92.701236   80.559319    73.899118   69.921451          NaN                                                              NaN NaN                  0.0775  312.211636    201.569598  120.010631      106.879208   94.127575   82.094856    75.532876   71.641225          NaN                                                              NaN NaN                    0.08  313.363655    202.763943  121.327594      108.245258   95.565208   83.644007    77.181622   73.376457          NaN                                                              NaN NaN                  0.0825  314.518234    203.962517  122.652625      109.620727   97.014036   85.206565    78.845013   75.126660          NaN                                                              NaN NaN                   0.085  315.675374    205.165313  123.985689      111.005556   98.473956   86.782323    80.522708   76.891348          NaN                                                              NaN NaN                  0.0875  316.835072    206.372327  125.326750       112.39969   99.944 865   88.371071    82.214364   78.670041          NaN                                                              NaN NaN                    0.09  317.997327    207.583552  126.675774       113.80307  101.426658   89.972596    83.919636   80.462262          NaN                                                              NaN NaN                  0.0925  319.162136    208.798983  128.032722      115.215634  102.919229   91.586683    85.638184   82.267543          NaN                                                              NaN NaN                   0.095  320.329497    210.018613  129.397558      116.637324  104.422468   93.213119    87.369666   84.085421          NaN                                                              NaN NaN                  0.0975   321.49941    211.242437  130.770242      118.068075  105.936266   94.851685    89.113742   85.915441          NaN                                                              NaN NaN                     0.1  322.671872    212.470447  132.150737      119.507826  107.460512   96.502165    90.870075   87.757157          NaN                                                              NaN NaN                  0.1025  323.846881    213.702638  133.539002      120.956512  108.995092   98.164339    92.638328   89.610130          NaN                                                              NaN NaN                   0.105  325.024435    214.939004  134.934997      122.414068  110.539892   99.837989    94.418171   91.473929          NaN                                                              NaN NaN                  0.1075  326.204532    216.179537  136.338680      123.880428  112.094798  101.522895    96.209272   93 .348136          NaN                                                              NaN NaN                    0.11  327.387171    217.424231  137.750011      125.355526  113.659693  103.218839    98.011308   95.232340          NaN                                                              NaN NaN                  0.1125  328.572349    218.673079  139.168947      126.839292  115.234460  104.925601    99.823955   97.126139          NaN                                                              NaN NaN                   0.115  329.760064    219.926074  140.595444      128.331658  116.818981  106.642963   101.646896   99.029143          NaN                                                              NaN NaN                  0.1175  330.950315    221.183209  142.029459      129.832556  118.413136  108.370706   103.479819  100.940973          NaN                                                              NaN NaN                    0.12  332.143098    222.444477  143.470948      131.341914  120.016806  110.108613   105.322414  102.861260          NaN                                                              NaN NaN                  0.1225  333.338413     223.70987  144.919867      132.859662  121.629871  111.856468   107.174379  104.789643          NaN                                                              NaN NaN                   0.125  334.536256    224.979382  146.376169      134.385728  123.252208  113.614055   109.035414  106.725776          NaN                                                              NaN NaN                  0.1275  335.736626    226.253005  147.839809       135.92004  124.883698  115.381160   110.905227  108.669321          NaN                                                              NaN NaN                    0.13    336.93952     227.53073  149.310740      137.462524  126.524217  117.157571   112.783530  110.619952          NaN                                                              NaN NaN                  0.1325  338.144937    228.812552  150.788916      139.013108  128.173643  118.943077   114.670043  112.577352          NaN                                                              NaN NaN                   0.135  339.352873     230.09846  152.274289      140.571716  129.831854  120.737468   116.564488  114.541218          NaN                                                              NaN NaN                  0.1375  340.563328    231.388449  153.766812      142.138275  131.498727  122.540538   118.466597  116.511253          NaN                                                              NaN NaN                    0.14  341.776298    232.682508  155.266435      143.712708  133.174139  124.352081   120.376104  118.487175          NaN                                                              NaN NaN                  0.1425  342.991781    233.980632  156.773111      145.294942  134.857966  126.171894   122.292753  120.468709          NaN                                                              NaN NaN                   0.145  344.209774    235.282811  158.286789      146.884899  136.550086  127.999777   124.216292  122.455593          NaN                                                              NaN NaN                  0.1475  345.430277    236.589037  159.807422      148.482502  138.250376  129.835530   126.146474  124.447572          NaN                                                              NaN NaN                    0.15  346.653285    237.899301  161.334 957      150.087677  139.958712  131.678958   128.083061  126.444402          NaN                                                              NaN NaN                  0.1525  347.878797    239.213595  162.869346      151.700344  141.674972  133.529868   130.025820  128.445850          NaN                                                              NaN NaN                   0.155   349.10681    240.531911  164.410537      153.320427  143.399034  135.388069   131.974524  130.451692          NaN                                                              NaN NaN                  0.1575  350.337322    241.854239  165.958479      154.947848  145.130775  137.253373   133.928951  132.461710          NaN                                                              NaN NaN                    0.16   351.57033    243.180571  167.513121       156.58253  146.870074  139.125594   135.888889  134.475700          NaN                                                              NaN Sheet: Wedding budget Unnamed: 0                 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  Unnamed: 5                      Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  Unnamed: 10  Unnamed: 11 Unnamed: 12           Unnamed: 13 Unnamed: 14 Unnamed: 15   Unnamed: 16 Unnamed: 17 NaN               PARTY BUDGET        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                    SUMMARY  ESTIMATED     ACTUAL OVER/UNDER         NaN      SUMMARY OF ACTUAL EXPENSES        NaN        NaN        NaN          NaN          NaN        Rank              Category   Estimated      Actual Ranking Value         Ref NaN                  Reception       5500       5300       -200         NaN                             NaN        NaN        NaN        NaN          NaN          NaN          10               Apparel           0           0        0.0006           1 NaN                Photography       2950       2950          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           9           Decorations           0           0        0.0007           2 NaN             Transportation       1450       1400        -50         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           4                 Gifts        1400        1300     1300.0008           3 NaN                      Gifts       1400       1300       -100         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           6               Flowers         800         800      800.0009           4 NaN                      Music       1200       1250         50         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           5                  Music        1200        1250      1250.001           5 NaN                    Flowers        800        800          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           2           Photography        2950        2950     2950.0011           6 NaN      Stationery / Printing        400        400          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           1             Reception        5500        5300     5300.0012           7 NaN             Other Expenses        300        300          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           7 Stationery / Printing         400         400      400.0013           8 NaN                Decorations          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           3        Transportation        1450        1400     1400.0014           9 NaN                    Apparel          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           8        Other Expenses         300         300      300.0015          10 NaN               Total budget      14000      13700         10         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN    NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                    APPAREL  ESTIMATED     ACTUAL OVER/UNDER         NaN                     DECORATIONS  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN            Engagement ring       5000       5000          0         NaN            B ows for church pews       1800       1800          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN              Wedding rings          0          0          0         NaN              Table centerpieces        500        500          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN  Tux, suit, and/or dresses          0          0          0         NaN                         Candles          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN             Veil/headpiece          0          0          0         NaN                        Lighting          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                      Shoes          0          0          0         NaN                        Balloons          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                    Jewelry          0          0          0         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                Accessories          0          0          0         NaN           Total for decorations          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Others          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN          Total for apparel          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                         FLOWERS  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                        Bouquets        800        800          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                      GIFTS  ESTIMATED     ACTUAL OVER/UNDER         NaN                    Boutonnières          0           0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                 Attendants       1200       1100       -100         NaN                        Corsages          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                    Parents        200        200          0         NaN                        Ceremony          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN Readers/other participants          0          0          0         NaN                       Reception          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Others          0          0          0         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN            Total for gifts       1400       1300       -100         NaN               Total for flowers        800        800          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     MUSIC   ESTIMATED     ACTUAL OVER/UNDER         NaN                     PHOTOGRAPHY  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN     Musicians for ceremony       1200       1250         50         NaN                         Formals       1200       1200          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN      Band/DJ for reception          0          0          0         NaN                        Candid's        800        800          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Others          0          0          0         NaN                    Extra prints          0          0          0          NaN           NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN            Total for music       1200       1250         50         NaN                    Photo albums          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                     Videography        950        950          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                  RECEPTION  ESTIMATED     ACTUAL OVER/UNDER         NaN           Total for photography       2950       2950          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN             Room/hall fees       5500       5300       -200         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN          Tables and chairs          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                       Food          0          0          0         NaN           STATIONERY / PRINTING  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Drinks          0          0          0         NaN                     Invitations        400        400          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Linens          0          0          0         NaN                   Announcements          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                       Cake          0          0          0         NaN                 Thank-You cards          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Favors          0          0          0         NaN             Personal stationery          0          0          0          NaN          NaN         NaN                   NaN          NaN         NaN           NaN         NaN NaN       Staff and gratuities          0          0          0         NaN                      Guest book          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Others          0          0          0         NaN                        Programs          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN        Total for reception       5500       5300       -200         NaN               Reception napkins          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                      Matchbooks          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                     Calligraphy          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN             OTHER EXPENSES  ESTIMATED     ACTUAL OVER/UNDER         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                  Officiant        300        300          0         NaN Total for stationery / printing        400        400          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN   Church/ceremony site fee          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN        Wedding coordinator          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN           Rehearsal dinner          0          0          0         NaN                  TRANSPORTATION  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN           Engagement party          0          0          0         NaN             Limousines/trolleys        900        900          0          NaN          NaN          NaN                   NaN         NaN         NaN           NaN         NaN NaN                    Showers          0          0          0         NaN                         Parking        200        200          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN      Bachelor/ette parties          0          0          0         NaN                           Taxis        350        300        -50          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                Hotel rooms          0          0          0         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN   Total for other expenses        300        300          0         NaN        Total for transportation       1450       1400        -50          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN"""


# Create a ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Score
scores = scorer.score(reference_summary, generated_summary)

# Print results
print("ROUGE Scores PEGASUS with TextRank:")
for metric, score in scores.items():
    print(f"{metric}: Precision: {score.precision:.4f}, Recall: {score.recall:.4f}, F1: {score.fmeasure:.4f}")


ROUGE Scores PEGASUS with TextRank:
rouge1: Precision: 0.0089, Recall: 0.1647, F1: 0.0169
rouge2: Precision: 0.0006, Recall: 0.0118, F1: 0.0012
rougeL: Precision: 0.0057, Recall: 0.1059, F1: 0.0109


In [24]:
#Comparing PEGASUS with Luhn 
from rouge_score import rouge_scorer

# Define your summaries here
reference_summary = """The following table lists the outstanding loans of the Republic of Ireland at the end of the first quarter of the current financial year, as reported by the Irish Central Bank (ICB). Match reports and team news for Saturday's Premier League game between Arsenal and Tottenham at Emirates Stadium in London, as Arsene Wenger's side take on Mauricio Pochettino's side. Here is a list of the key raw materials used in the production of NaN NaN, as reported by the US Geological Survey (USGS) in the third quarter of 2016. All photographs courtesy of AFP, EPA, Getty Images, Reuters, Getty Images and Reuters news agency.''Source: AFP, EPA, Getty Images and Reuters news agency Find out more about the events at this year's White House Correspondents' Association dinner at the Washington Hilton Hotel in Washington, DC, on Saturday, April 16. Here is a breakdown of some of the costs involved in getting married in Dublin, as compiled by the city's Department of Tourism and Sport."""
generated_summary = """Sheet: Loan Schedule Unnamed: 0                  Unnamed: 1                 Unnamed: 2         Unnamed: 3          Unnamed: 4     Unnamed: 5                   Unnamed: 6  Unnamed: 7     Unnamed: 8      Unnamed: 9          Unnamed: 10 NaN                         NaN Loan Amortisation Schedule                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN NaN                         NaN                        NaN                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN NaN                Enter Values                        NaN                NaN                 NaN            NaN                 Loan Summary         NaN            NaN             NaN                  NaN NaN                 Loan amount                        NaN                NaN                5000            NaN            Scheduled payment         NaN     425.749521             NaN                  NaN NaN        Annual interest rate                        NaN                NaN                0.04            NaN Scheduled number of payments         NaN             12             NaN                  NaN NaN        Loan period in years                        NaN                NaN                   1            NaN    Actual number of payments         NaN             10             NaN                  NaN NaN Number of payments per year                        NaN                NaN                  12            NaN         Total early payments         NaN            900             NaN                  NaN NaN          Start date of loan                        NaN                NaN 2025-04-07 00:00:00            NaN               Total interest         NaN      89.621486             NaN                  NaN NaN                         NaN                        NaN                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN NaN     Optional extra payments                        NaN                NaN                 100            NaN                  Lender name         NaN Woodgrove Bank             NaN                  NaN NaN                         NaN                        NaN                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN NaN              Payment Number              Payment\nDate Beginning\nBalance   Scheduled Payment Extra\nPayment               Total\nPayment   Principal       Interest Ending\nBalance Cumulative\nInterest NaN                           1        2025-04-07 00:00:00               5000          425.749521            100                   525.749521  509.082854      16.666667     4490.917146            16.666667 NaN                           2        2025-05-07 00: 00:00        4490.917146          425.749521            100                   525.749521  510.779797      14.969724     3980.137349             31.63639 NaN                           3        2025-06-07 00:00:00        3980.137349          425.749521            100                   525.749521  512.482396      13.267124     3467.654952            44.903515 NaN                           4        2025-07-07 00:00:00        3467.654952          425.749521            100                   525.749521  514.190671       11.55885     2953.464281            56.462365 NaN                           5        2025-08-07 00:00:00        2953.464281          425.749521            100                   525.749521   515.90464       9.844881     2437.559641            66.307246 NaN                           6        2025-09-07 00:00:00        2437.559641          425.749521            100                   525.749521  517.624322       8.125199     1919.935319            74.432445 NaN                           7        2025-10-07 00:00:00        1919.935319          425.749521            100                   525.749521  519.349737       6.399784     1400.585582            80.832229 NaN                           8        2025-11-07 00:00:00        1400.585582          425.749521            100                   525.749521  521.080902       4.668619       879.50468            85.500848 NaN                           9        2025-12-07 00:00:00          879.50468          425.749521            100                   525.749521  522.817839        2.931682      356.686841             88.43253 NaN                          10        2026-01-07 00:00:00         356.686841          425.749521              0                   356.686841  355.497885       1.188956               0            89.621486 Sheet: LOAN ANALYSIS Unnamed: 0 LOAN ANALYSIS WORKSHEET  Unnamed: 2    Unnamed: 3  Unnamed: 4      Unnamed: 5  Unnamed: 6  Unnamed: 7   Unnamed: 8  Unnamed: 9  Unnamed: 10                                                      Unnamed: 11 NaN           LOAN ANALYSIS         NaN           NaN         NaN             NaN         NaN         NaN          NaN         NaN          NaN Rate slicer to filter table data based on Rates is in this cell.
NaN           INTEREST RATE         NaN          0.05         NaN MONTHLY PAYMENT         NaN         NaN   106.065515         NaN          NaN                                                              NaN NaN           YEARS OF LOAN         NaN            10         NaN   TOTAL PAYMENT         NaN         NaN 12727.861829         NaN          NaN                                                              NaN NaN             LOAN AMOUNT         NaN         10000         NaN  TOTAL INTEREST         NaN         NaN  2727.861829         NaN          NaN                                                              NaN NaN            PAYMENTS DUE         NaN End of Period         NaN             NaN         NaN         NaN          NaN         NaN          NaN                                                              NaN NaN                     NaN       YEARS           NaN         NaN             NaN         NaN         NaN          NaN         NaN          NaN                                                              NaN NaN                    RATE           3             5   10.000000              12   15.000000   20.000000    25.000000   30.000000          NaN                                                              NaN NaN                    0.02  286.425787    175.277601   92.013454       78.168369   64.350870   50.588334    42.385434   36.961947          NaN                                                              NaN NaN                  0.0225  287.518475    176.373448   93.137372       79.305498   65.508477   51.780829    43.613070   38.224610          NaN                                                              NaN NaN                   0.025  288.613757    177.473616   94.269902       80.452938   66.678921   52.990289    44.861673    39.512090          NaN                                                              NaN NaN                  0.0275  289.711631    178.578103   95.411031        81.61067   67.862164   54.216631    46.131086   40.824118          NaN                                                              NaN NaN                    0.03  290.812096    179.686907   96.560745       82.778669   69.058164   55.459760    47.421131   42.160403          NaN                                                              NaN NaN                  0.0325  291.915152    180.800023   97.719029       83.956912   70.266877   56.719576    48.731623   43.520632          NaN                                                              NaN NaN                   0.035  293.020797     181.91745   98.885867       85.145374   71.488254   57.995972    50.062357   44.904469          NaN                                                              NaN NaN                  0.0375   294.12903    183.039183  100.061243       86.344025   72.722244   59.288831    51.413120   46.311559          NaN                                                              NaN NaN                    0.04   295.23985    184.165221  101.245138       87.552837   73.968793   60.598033    52.783684   47.741530          NaN                                                              NaN NaN                  0.0425  296.353255    185.295558  102.437533       88.771778   75.227841   61.923447    54.173810   49.193989          NaN                                                              NaN NaN                   0.045  297.469245    186.430192  103.638409       90.000816   76.499329   63.264938    55.583248   50.668531          NaN                                                              NaN NaN                  0 .0475  298.587817     187.56912  104.847743       91.239916   77.783192   64.622363    57.011736   52.164734          NaN                                                              NaN NaN                    0.05  299.708971    188.712336  106.065515       92.489041   79.079363   65.995574    58.459004   53.682162          NaN                                                              NaN NaN                  0.0525  300.832705    189.859838  107.291701       93.748155   80.387772   67.384417    59.924772   55.220370          NaN                                                              NaN NaN                   0.055  301.959018    191.011622  108.526278       95.017217   81.708345   68.788731    61.408749   56.778900          NaN                                                              NaN NaN                  0.0575  303.087908    192.167682  109.769220       96.296187   83.041009   70.208351    62.910640   58.357286          NaN                                                              NaN NaN                    0.06  304.219375    193.328015  111.020502       97.585021   84.385683   71.643106    64.430140   59.955053          NaN                                                              NaN NaN                  0.0625  305.353415    194.492617  112.280097       98.883677   85.742287   73.092820    65.966938   61.571720          NaN                                                              NaN NaN                   0.065  306.490029    195.661482  113.547977      100.192109   87.110737   74.557314    67.520716   63.206802          NaN                                                              NaN NaN                  0.0675  307.629214    196.834607   114.824114      101.510269   88.490946   76.036401    69.091153   64.859810          NaN                                                              NaN NaN                    0.07  308.770969    198.011985  116.108479       102.83811   89.882827   77.529894    70.677920   66.530250          NaN                                                              NaN NaN                  0.0725  309.915292    199.193614  117.401041      104.175581   91.286288   79.037598    72.280686   68.217628          NaN                                                              NaN NaN                   0.075  311.062182    200.379486  118.701769      105.522631   92.701236   80.559319    73.899118   69.921451          NaN                                                              NaN NaN                  0.0775  312.211636    201.569598  120.010631      106.879208   94.127575   82.094856    75.532876   71.641225          NaN                                                              NaN NaN                    0.08  313.363655    202.763943  121.327594      108.245258   95.565208   83.644007    77.181622   73.376457          NaN                                                              NaN NaN                  0.0825  314.518234    203.962517  122.652625      109.620727   97.014036   85.206565    78.845013   75.126660          NaN                                                              NaN NaN                   0.085  315.675374    205.165313  123.985689      111.005556   98.473956   86.782323    80.522708   76.891348          NaN                                                              NaN NaN                  0.0875  316.835072    206.372327  125.326750       112.39969   99.944 865   88.371071    82.214364   78.670041          NaN                                                              NaN NaN                    0.09  317.997327    207.583552  126.675774       113.80307  101.426658   89.972596    83.919636   80.462262          NaN                                                              NaN NaN                  0.0925  319.162136    208.798983  128.032722      115.215634  102.919229   91.586683    85.638184   82.267543          NaN                                                              NaN NaN                   0.095  320.329497    210.018613  129.397558      116.637324  104.422468   93.213119    87.369666   84.085421          NaN                                                              NaN NaN                  0.0975   321.49941    211.242437  130.770242      118.068075  105.936266   94.851685    89.113742   85.915441          NaN                                                              NaN NaN                     0.1  322.671872    212.470447  132.150737      119.507826  107.460512   96.502165    90.870075   87.757157          NaN                                                              NaN NaN                  0.1025  323.846881    213.702638  133.539002      120.956512  108.995092   98.164339    92.638328   89.610130          NaN                                                              NaN NaN                   0.105  325.024435    214.939004  134.934997      122.414068  110.539892   99.837989    94.418171   91.473929          NaN                                                              NaN NaN                  0.1075  326.204532    216.179537  136.338680      123.880428  112.094798  101.522895    96.209272   93 .348136          NaN                                                              NaN NaN                    0.11  327.387171    217.424231  137.750011      125.355526  113.659693  103.218839    98.011308   95.232340          NaN                                                              NaN NaN                  0.1125  328.572349    218.673079  139.168947      126.839292  115.234460  104.925601    99.823955   97.126139          NaN                                                              NaN NaN                   0.115  329.760064    219.926074  140.595444      128.331658  116.818981  106.642963   101.646896   99.029143          NaN                                                              NaN NaN                  0.1175  330.950315    221.183209  142.029459      129.832556  118.413136  108.370706   103.479819  100.940973          NaN                                                              NaN NaN                    0.12  332.143098    222.444477  143.470948      131.341914  120.016806  110.108613   105.322414  102.861260          NaN                                                              NaN NaN                  0.1225  333.338413     223.70987  144.919867      132.859662  121.629871  111.856468   107.174379  104.789643          NaN                                                              NaN NaN                   0.125  334.536256    224.979382  146.376169      134.385728  123.252208  113.614055   109.035414  106.725776          NaN                                                              NaN NaN                  0.1275  335.736626    226.253005  147.839809       135.92004  124.883698  115.381160   110.905227  108.669321          NaN                                                              NaN NaN                    0.13    336.93952     227.53073  149.310740      137.462524  126.524217  117.157571   112.783530  110.619952          NaN                                                              NaN NaN                  0.1325  338.144937    228.812552  150.788916      139.013108  128.173643  118.943077   114.670043  112.577352          NaN                                                              NaN NaN                   0.135  339.352873     230.09846  152.274289      140.571716  129.831854  120.737468   116.564488  114.541218          NaN                                                              NaN NaN                  0.1375  340.563328    231.388449  153.766812      142.138275  131.498727  122.540538   118.466597  116.511253          NaN                                                              NaN NaN                    0.14  341.776298    232.682508  155.266435      143.712708  133.174139  124.352081   120.376104  118.487175          NaN                                                              NaN NaN                  0.1425  342.991781    233.980632  156.773111      145.294942  134.857966  126.171894   122.292753  120.468709          NaN                                                              NaN NaN                   0.145  344.209774    235.282811  158.286789      146.884899  136.550086  127.999777   124.216292  122.455593          NaN                                                              NaN NaN                  0.1475  345.430277    236.589037  159.807422      148.482502  138.250376  129.835530   126.146474  124.447572          NaN                                                              NaN NaN                    0.15  346.653285    237.899301  161.334 957      150.087677  139.958712  131.678958   128.083061  126.444402          NaN                                                              NaN NaN                  0.1525  347.878797    239.213595  162.869346      151.700344  141.674972  133.529868   130.025820  128.445850          NaN                                                              NaN NaN                   0.155   349.10681    240.531911  164.410537      153.320427  143.399034  135.388069   131.974524  130.451692          NaN                                                              NaN NaN                  0.1575  350.337322    241.854239  165.958479      154.947848  145.130775  137.253373   133.928951  132.461710          NaN                                                              NaN NaN                    0.16   351.57033    243.180571  167.513121       156.58253  146.870074  139.125594   135.888889  134.475700          NaN                                                              NaN Sheet: Wedding budget Unnamed: 0                 Unnamed: 1 Unnamed: 2 Unnamed: 3 Unnamed: 4  Unnamed: 5                      Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  Unnamed: 10  Unnamed: 11 Unnamed: 12           Unnamed: 13 Unnamed: 14 Unnamed: 15   Unnamed: 16 Unnamed: 17 NaN               PARTY BUDGET        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                    SUMMARY  ESTIMATED     ACTUAL OVER/UNDER         NaN      SUMMARY OF ACTUAL EXPENSES        NaN        NaN        NaN          NaN          NaN        Rank              Category   Estimated      Actual Ranking Value         Ref NaN                  Reception       5500       5300       -200         NaN                             NaN        NaN        NaN        NaN          NaN          NaN          10               Apparel           0           0        0.0006           1 NaN                Photography       2950       2950          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           9           Decorations           0           0        0.0007           2 NaN             Transportation       1450       1400        -50         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           4                 Gifts        1400        1300     1300.0008           3 NaN                      Gifts       1400       1300       -100         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           6               Flowers         800         800      800.0009           4 NaN                      Music       1200       1250         50         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           5                  Music        1200        1250      1250.001           5 NaN                    Flowers        800        800          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           2           Photography        2950        2950     2950.0011           6 NaN      Stationery / Printing        400        400          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           1             Reception        5500        5300     5300.0012           7 NaN             Other Expenses        300        300          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           7 Stationery / Printing         400         400      400.0013           8 NaN                Decorations          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           3        Transportation        1450        1400     1400.0014           9 NaN                    Apparel          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           8        Other Expenses         300         300      300.0015          10 NaN               Total budget      14000      13700         10         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                    APPAREL  ESTIMATED     ACTUAL OVER/UNDER         NaN                     DECORATIONS  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN            Engagement ring       5000       5000          0         NaN            B ows for church pews       1800       1800          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN              Wedding rings          0          0          0         NaN              Table centerpieces        500        500          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN  Tux, suit, and/or dresses          0          0          0         NaN                         Candles          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN             Veil/headpiece          0          0          0         NaN                        Lighting          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                      Shoes          0          0          0         NaN                        Balloons          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                    Jewelry          0          0          0         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                Accessories          0          0          0         NaN           Total for decorations          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Others          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN          Total for apparel          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                         FLOWERS  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                        Bouquets        800        800          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                      GIFTS  ESTIMATED     ACTUAL OVER/UNDER         NaN                    Boutonnières          0           0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                 Attendants       1200       1100       -100         NaN                        Corsages          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                    Parents        200        200          0         NaN                        Ceremony          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN Readers/other participants          0          0          0         NaN                       Reception          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Others          0          0          0         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN            Total for gifts       1400       1300       -100         NaN               Total for flowers        800        800          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     MUSIC   ESTIMATED     ACTUAL OVER/UNDER         NaN                     PHOTOGRAPHY  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN     Musicians for ceremony       1200       1250         50         NaN                         Formals       1200       1200          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN      Band/DJ for reception          0          0          0         NaN                        Candid's        800        800          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Others          0          0          0         NaN                    Extra prints          0          0          0          NaN           NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN            Total for music       1200       1250         50         NaN                    Photo albums          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                     Videography        950        950          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                  RECEPTION  ESTIMATED     ACTUAL OVER/UNDER         NaN           Total for photography       2950       2950          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN             Room/hall fees       5500       5300       -200         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN          Tables and chairs          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                       Food          0          0          0         NaN           STATIONERY / PRINTING  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Drinks          0          0          0         NaN                     Invitations        400        400          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Linens          0          0          0         NaN                   Announcements          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                       Cake          0          0          0         NaN                 Thank-You cards          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Favors          0          0          0         NaN             Personal stationery          0          0          0          NaN          NaN         NaN                   NaN          NaN         NaN           NaN         NaN NaN       Staff and gratuities          0          0          0         NaN                      Guest book          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                     Others          0          0          0         NaN                        Programs          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN        Total for reception       5500       5300       -200         NaN               Reception napkins          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                      Matchbooks          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                        NaN        NaN        NaN        NaN         NaN                     Calligraphy          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN             OTHER EXPENSES  ESTIMATED     ACTUAL OVER/UNDER         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                  Officiant        300        300          0         NaN Total for stationery / printing        400        400          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN   Church/ceremony site fee          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN        Wedding coordinator          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN           Rehearsal dinner          0          0          0         NaN                  TRANSPORTATION  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN           Engagement party          0          0          0         NaN             Limousines/trolleys        900        900          0          NaN          NaN          NaN                   NaN         NaN         NaN           NaN         NaN NaN                    Showers          0          0          0         NaN                         Parking        200        200          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN      Bachelor/ette parties          0          0          0         NaN                           Taxis        350        300        -50          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN                Hotel rooms          0          0          0         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN NaN   Total for other expenses        300        300          0         NaN        Total for transportation       1450       1400        -50          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN """


# Create a ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Score
scores = scorer.score(reference_summary, generated_summary)

# Print results
print("ROUGE Scores PEGASUS with Luhn:")
for metric, score in scores.items():
    print(f"{metric}: Precision: {score.precision:.4f}, Recall: {score.recall:.4f}, F1: {score.fmeasure:.4f}")


ROUGE Scores PEGASUS with Luhn:
rouge1: Precision: 0.0089, Recall: 0.1647, F1: 0.0169
rouge2: Precision: 0.0006, Recall: 0.0118, F1: 0.0012
rougeL: Precision: 0.0057, Recall: 0.1059, F1: 0.0109


In [23]:
#Comparing PEGASUS with SpaCy (Custom) 
from rouge_score import rouge_scorer

# Define your summaries here
reference_summary = """The following table lists the outstanding loans of the Republic of Ireland at the end of the first quarter of the current financial year, as reported by the Irish Central Bank (ICB). Match reports and team news for Saturday's Premier League game between Arsenal and Tottenham at Emirates Stadium in London, as Arsene Wenger's side take on Mauricio Pochettino's side. Here is a list of the key raw materials used in the production of NaN NaN, as reported by the US Geological Survey (USGS) in the third quarter of 2016. All photographs courtesy of AFP, EPA, Getty Images, Reuters, Getty Images and Reuters news agency.''Source: AFP, EPA, Getty Images and Reuters news agency Find out more about the events at this year's White House Correspondents' Association dinner at the Washington Hilton Hotel in Washington, DC, on Saturday, April 16. Here is a breakdown of some of the costs involved in getting married in Dublin, as compiled by the city's Department of Tourism and Sport."""
generated_summary = """Unnamed: 15   Unnamed: 16 Unnamed: 17
        NaN               PARTY BUDGET        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                    SUMMARY  ESTIMATED     ACTUAL OVER/UNDER         NaN      SUMMARY OF ACTUAL EXPENSES        NaN        NaN        NaN          NaN          NaN        Rank              Category   Estimated      Actual Ranking Value         Ref
        NaN                  Reception       5500       5300       -200         NaN                             NaN        NaN        NaN        NaN          NaN          NaN          10               Apparel           0           0        0.0006           1
        NaN                Photography       2950       2950          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           9           Decorations           0           0        0.0007           2
        NaN             Transportation       1450       1400        -50         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           4                 Gifts        1400        1300     1300.0008           3
        NaN                      Gifts       1400       1300       -100         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           6               Flowers         800         800      800.0009           4
        NaN                      Music       1200       1250         50         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           5                  Music        1200        1250      1250.001           5
        NaN                    Flowers        800        800          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           2           Photography        2950        2950     2950.0011           6
        NaN      Stationery / Printing        400        400          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           1             Reception        5500        5300     5300.0012           7
        NaN             Other Expenses        300        300          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           7 Stationery / Printing         400         400      400.0013           8
        NaN                Decorations          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           3        Transportation        1450        1400     1400.0014           9
        NaN                    Apparel          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN           8        Other Expenses         300         300      300.0015          10
        NaN               Total budget      14000      13700         10         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                    APPAREL  ESTIMATED     ACTUAL OVER/UNDER         NaN                     DECORATIONS  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN            Engagement ring       5000       5000          0         NaN            B ows for church pews       1800       1800          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN              Wedding rings          0          0          0         NaN              Table centerpieces        500        500          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN  Tux, suit, and/or dresses          0          0          0         NaN                         Candles          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN             Veil/headpiece          0          0          0         NaN                        Lighting          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                      Shoes          0          0          0         NaN                        Balloons          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                    Jewelry          0          0          0         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                Accessories          0          0          0         NaN           Total for decorations          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                     Others          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN          Total for apparel          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                         FLOWERS  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                        Bouquets        800        800          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                      GIFTS  ESTIMATED     ACTUAL OVER/UNDER         NaN                    Boutonnières          0           0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                 Attendants       1200       1100       -100         NaN                        Corsages          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                    Parents        200        200          0         NaN                        Ceremony          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN Readers/other participants          0          0          0         NaN                       Reception          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                     Others          0          0          0         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN            Total for gifts       1400       1300       -100         NaN               Total for flowers        800        800          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                     MUSIC   ESTIMATED     ACTUAL OVER/UNDER         NaN                     PHOTOGRAPHY  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN     Musicians for ceremony       1200       1250         50         NaN                         Formals       1200       1200          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN      Band/DJ for reception          0          0          0         NaN                        Candid's        800        800          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                     Others          0          0          0         NaN                    Extra prints          0          0          0          NaN           NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN            Total for music       1200       1250         50         NaN                    Photo albums          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                     Videography        950        950          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                  RECEPTION  ESTIMATED     ACTUAL OVER/UNDER         NaN           Total for photography       2950       2950          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN             Room/hall fees       5500       5300       -200         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN          Tables and chairs          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                       Food          0          0          0         NaN           STATIONERY / PRINTING  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                     Drinks          0          0          0         NaN                     Invitations        400        400          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                     Linens          0          0          0         NaN                   Announcements          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                       Cake          0          0          0         NaN                 Thank-You cards          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                     Favors          0          0          0         NaN             Personal stationery          0          0          0          NaN          NaN         NaN                   NaN          NaN         NaN           NaN         NaN
        NaN       Staff and gratuities          0          0          0         NaN                      Guest book          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                     Others          0          0          0         NaN                        Programs          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN        Total for reception       5500       5300       -200         NaN               Reception napkins          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                      Matchbooks          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                        NaN        NaN        NaN        NaN         NaN                     Calligraphy          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN             OTHER EXPENSES  ESTIMATED     ACTUAL OVER/UNDER         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                  Officiant        300        300          0         NaN Total for stationery / printing        400        400          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN   Church/ceremony site fee          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN        Wedding coordinator          0          0          0         NaN                             NaN        NaN        NaN        NaN          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN           Rehearsal dinner          0          0          0         NaN                  TRANSPORTATION  ESTIMATED     ACTUAL OVER/UNDER          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN           Engagement party          0          0          0         NaN             Limousines/trolleys        900        900          0          NaN          NaN          NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                    Showers          0          0          0         NaN                         Parking        200        200          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN      Bachelor/ette parties          0          0          0         NaN                           Taxis        350        300        -50          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN                Hotel rooms          0          0          0         NaN                          Others          0          0          0          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
        NaN   Total for other expenses        300        300          0         NaN        Total for transportation       1450       1400        -50          NaN          NaN         NaN                   NaN         NaN         NaN           NaN         NaN
.0475  298.587817     187.56912  104.847743       91.239916   77.783192   64.622363    57.011736   52.164734          NaN                                                              NaN
        NaN                    0.05  299.708971    188.712336  106.065515       92.489041   79.079363   65.995574    58.459004   53.682162          NaN                                                              NaN
        NaN                  0.0525  300.832705    189.859838  107.291701       93.748155   80.387772   67.384417    59.924772   55.220370          NaN                                                              NaN
        NaN                   0.055  301.959018    191.011622  108.526278       95.017217   81.708345   68.788731    61.408749   56.778900          NaN                                                              NaN
        NaN                  0.0575  303.087908    192.167682  109.769220       96.296187   83.041009   70.208351    62.910640   58.357286          NaN                                                              NaN
        NaN                    0.06  304.219375    193.328015  111.020502       97.585021   84.385683   71.643106    64.430140   59.955053          NaN                                                              NaN
        NaN                  0.0625  305.353415    194.492617  112.280097       98.883677   85.742287   73.092820    65.966938   61.571720          NaN                                                              NaN
        NaN                   0.065  306.490029    195.661482  113.547977      100.192109   87.110737   74.557314    67.520716   63.206802          NaN                                                              NaN
        NaN                  0.0675  307.629214    196.834607   114.824114      101.510269   88.490946   76.036401    69.091153   64.859810          NaN                                                              NaN
        NaN                    0.07  308.770969    198.011985  116.108479       102.83811   89.882827   77.529894    70.677920   66.530250          NaN                                                              NaN
        NaN                  0.0725  309.915292    199.193614  117.401041      104.175581   91.286288   79.037598    72.280686   68.217628          NaN                                                              NaN
        NaN                   0.075  311.062182    200.379486  118.701769      105.522631   92.701236   80.559319    73.899118   69.921451          NaN                                                              NaN
        NaN                  0.0775  312.211636    201.569598  120.010631      106.879208   94.127575   82.094856    75.532876   71.641225          NaN                                                              NaN
        NaN                    0.08  313.363655    202.763943  121.327594      108.245258   95.565208   83.644007    77.181622   73.376457          NaN                                                              NaN
        NaN                  0.0825  314.518234    203.962517  122.652625      109.620727   97.014036   85.206565    78.845013   75.126660          NaN                                                              NaN
        NaN                   0.085  315.675374    205.165313  123.985689      111.005556   98.473956   86.782323    80.522708   76.891348          NaN                                                              NaN
        NaN                  0.0875  316.835072    206.372327  125.326750       112.39969   99.944 865   88.371071    82.214364   78.670041          NaN                                                              NaN
        NaN                    0.09  317.997327    207.583552  126.675774       113.80307  101.426658   89.972596    83.919636   80.462262          NaN                                                              NaN
        NaN                  0.0925  319.162136    208.798983  128.032722      115.215634  102.919229   91.586683    85.638184   82.267543          NaN                                                              NaN
        NaN                   0.095  320.329497    210.018613  129.397558      116.637324  104.422468   93.213119    87.369666   84.085421          NaN                                                              NaN
        NaN                  0.0975   321.49941    211.242437  130.770242      118.068075  105.936266   94.851685    89.113742   85.915441          NaN                                                              NaN
        NaN                     0.1  322.671872    212.470447  132.150737      119.507826  107.460512   96.502165    90.870075   87.757157          NaN                                                              NaN
        NaN                  0.1025  323.846881    213.702638  133.539002      120.956512  108.995092   98.164339    92.638328   89.610130          NaN                                                              NaN
        NaN                   0.105  325.024435    214.939004  134.934997      122.414068  110.539892   99.837989    94.418171   91.473929          NaN                                                              NaN
        NaN                  0.1075  326.204532    216.179537  136.338680      123.880428  112.094798  101.522895    96.209272   93 .348136          NaN                                                              NaN
        NaN                    0.11  327.387171    217.424231  137.750011      125.355526  113.659693  103.218839    98.011308   95.232340          NaN                                                              NaN
        NaN                  0.1125  328.572349    218.673079  139.168947      126.839292  115.234460  104.925601    99.823955   97.126139          NaN                                                              NaN
        NaN                   0.115  329.760064    219.926074  140.595444      128.331658  116.818981  106.642963   101.646896   99.029143          NaN                                                              NaN
        NaN                  0.1175  330.950315    221.183209  142.029459      129.832556  118.413136  108.370706   103.479819  100.940973          NaN                                                              NaN
        NaN                    0.12  332.143098    222.444477  143.470948      131.341914  120.016806  110.108613   105.322414  102.861260          NaN                                                              NaN
        NaN                  0.1225  333.338413     223.70987  144.919867      132.859662  121.629871  111.856468   107.174379  104.789643          NaN                                                              NaN
        NaN                   0.125  334.536256    224.979382  146.376169      134.385728  123.252208  113.614055   109.035414  106.725776          NaN                                                              NaN
        NaN                  0.1275  335.736626    226.253005  147.839809       135.92004  124.883698  115.381160   110.905227  108.669321          NaN                                                              NaN
        NaN                    0.13    336.93952     227.53073  149.310740      137.462524  126.524217  117.157571   112.783530  110.619952          NaN                                                              NaN
        NaN                  0.1325  338.144937    228.812552  150.788916      139.013108  128.173643  118.943077   114.670043  112.577352          NaN                                                              NaN
        NaN                   0.135  339.352873     230.09846  152.274289      140.571716  129.831854  120.737468   116.564488  114.541218          NaN                                                              NaN
        NaN                  0.1375  340.563328    231.388449  153.766812      142.138275  131.498727  122.540538   118.466597  116.511253          NaN                                                              NaN
        NaN                    0.14  341.776298    232.682508  155.266435      143.712708  133.174139  124.352081   120.376104  118.487175          NaN                                                              NaN
        NaN                  0.1425  342.991781    233.980632  156.773111      145.294942  134.857966  126.171894   122.292753  120.468709          NaN                                                              NaN
        NaN                   0.145  344.209774    235.282811  158.286789      146.884899  136.550086  127.999777   124.216292  122.455593          NaN                                                              NaN
        NaN                  0.1475  345.430277    236.589037  159.807422      148.482502  138.250376  129.835530   126.146474  124.447572          NaN                                                              NaN
        NaN                    0.15  346.653285    237.899301  161.334 957      150.087677  139.958712  131.678958   128.083061  126.444402          NaN                                                              NaN
        NaN                  0.1525  347.878797    239.213595  162.869346      151.700344  141.674972  133.529868   130.025820  128.445850          NaN                                                              NaN
        NaN                   0.155   349.10681    240.531911  164.410537      153.320427  143.399034  135.388069   131.974524  130.451692          NaN                                                              NaN
        NaN                  0.1575  350.337322    241.854239  165.958479      154.947848  145.130775  137.253373   133.928951  132.461710          NaN                                                              NaN
        NaN                    0.16   351.57033    243.180571  167.513121       156.58253  146.870074  139.125594   135.888889  134.475700          NaN                                                              NaN Sheet: Wedding budget
 Unnamed: 0
Sheet: Loan Schedule
 Unnamed: 0                  Unnamed: 1                 Unnamed: 2         Unnamed: 3          Unnamed: 4     Unnamed: 5                   Unnamed: 6  Unnamed: 7     Unnamed: 8      Unnamed: 9          Unnamed: 10
        NaN                         NaN Loan Amortisation Schedule                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN
        NaN                         NaN                        NaN                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN
        NaN                Enter Values                        NaN                NaN                 NaN            NaN                 Loan Summary         NaN            NaN             NaN                  NaN
        NaN                 Loan amount                        NaN                NaN                5000            NaN            Scheduled payment         NaN     425.749521             NaN                  NaN
        NaN        Annual interest rate                        NaN                NaN                0.04            NaN Scheduled number of payments         NaN             12             NaN                  NaN
        NaN        Loan period in years                        NaN                NaN                   1            NaN    Actual number of payments         NaN             10             NaN                  NaN
        NaN Number of payments per year                        NaN                NaN                  12            NaN         Total early payments         NaN            900             NaN                  NaN
        NaN          Start date of loan                        NaN                NaN 2025-04-07 00:00:00            NaN               Total interest         NaN      89.621486             NaN                  NaN
        NaN                         NaN                        NaN                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN
        NaN     Optional extra payments                        NaN                NaN                 100            NaN                  Lender name         NaN Woodgrove Bank             NaN                  NaN
        NaN                         NaN                        NaN                NaN                 NaN            NaN                          NaN         NaN            NaN             NaN                  NaN
        NaN              Payment Number              Payment\nDate Beginning\nBalance   Scheduled Payment Extra\nPayment               Total\nPayment   Principal       Interest Ending\nBalance Cumulative\nInterest
        NaN                           1        2025-04-07 00:00:00               5000          425.749521            100                   525.749521  509.082854      16.666667     4490.917146            16.666667
        NaN                           2        2025-05-07 00: 00:00        4490.917146          425.749521            100                   525.749521  510.779797      14.969724     3980.137349             31.63639
        NaN                           3        2025-06-07 00:00:00        3980.137349          425.749521            100                   525.749521  512.482396      13.267124     3467.654952            44.903515
        NaN                           4        2025-07-07 00:00:00        3467.654952          425.749521            100                   525.749521  514.190671       11.55885     2953.464281            56.462365
        NaN                           5        2025-08-07 00:00:00        2953.464281          425.749521            100                   525.749521   515.90464       9.844881     2437.559641            66.307246
        NaN                           6        2025-09-07 00:00:00        2437.559641          425.749521            100                   525.749521  517.624322       8.125199     1919.935319            74.432445
        NaN                           7        2025-10-07 00:00:00        1919.935319          425.749521            100                   525.749521  519.349737       6.399784     1400.585582            80.832229
        NaN                           8        2025-11-07 00:00:00        1400.585582          425.749521            100                   525.749521  521.080902       4.668619       879.50468            85.500848
        NaN                           9        2025-12-07 00:00:00          879.50468          425.749521            100                   525.749521  522.817839        2.931682      356.686841             88.43253
        NaN                          10        2026-01-07 00:00:00         356.686841          425.749521              0                   356.686841  355.497885       1.188956               0            89.621486 Sheet: LOAN ANALYSIS
 Unnamed: 0 LOAN ANALYSIS WORKSHEET  Unnamed: 2    Unnamed: 3  Unnamed: 4      Unnamed: 5  Unnamed: 6  Unnamed: 7   Unnamed: 8  Unnamed: 9  Unnamed: 10                                                      Unnamed: 11
        NaN           LOAN ANALYSIS         NaN           NaN         NaN             NaN         NaN         NaN          NaN         NaN          NaN Rate slicer to filter table data based on Rates is in this cell.
NaN           INTEREST RATE         NaN          0.05         NaN MONTHLY PAYMENT         NaN         NaN   106.065515         NaN          NaN                                                              NaN
        NaN           YEARS OF LOAN         NaN            10         NaN   TOTAL PAYMENT         NaN         NaN 12727.861829         NaN          NaN                                                              NaN
        NaN             LOAN AMOUNT         NaN         10000         NaN  TOTAL INTEREST         NaN         NaN  2727.861829         NaN          NaN                                                              NaN
        NaN            PAYMENTS DUE         NaN End of Period         NaN             NaN         NaN         NaN          NaN         NaN          NaN                                                              NaN
        NaN                     NaN       YEARS           NaN         NaN             NaN         NaN         NaN          NaN         NaN          NaN                                                              NaN
        NaN                    RATE           3             5   10.000000              12   15.000000   20.000000    25.000000   30.000000          NaN                                                              NaN
        NaN                    0.02  286.425787    175.277601   92.013454       78.168369   64.350870   50.588334    42.385434   36.961947          NaN                                                              NaN
        NaN                  0.0225  287.518475    176.373448   93.137372       79.305498   65.508477   51.780829    43.613070   38.224610          NaN                                                              NaN
        NaN                   0.025  288.613757    177.473616   94.269902       80.452938   66.678921   52.990289    44.861673    39.512090          NaN                                                              NaN
        NaN                  0.0275  289.711631    178.578103   95.411031        81.61067   67.862164   54.216631    46.131086   40.824118          NaN                                                              NaN
        NaN                    0.03  290.812096    179.686907   96.560745       82.778669   69.058164   55.459760    47.421131   42.160403          NaN                                                              NaN
        NaN                  0.0325  291.915152    180.800023   97.719029       83.956912   70.266877   56.719576    48.731623   43.520632          NaN                                                              NaN
        NaN                   0.035  293.020797     181.91745   98.885867       85.145374   71.488254   57.995972    50.062357   44.904469          NaN                                                              NaN
        NaN                  0.0375   294.12903    183.039183  100.061243       86.344025   72.722244   59.288831    51.413120   46.311559          NaN                                                              NaN
        NaN                    0.04   295.23985    184.165221  101.245138       87.552837   73.968793   60.598033    52.783684   47.741530          NaN                                                              NaN
        NaN                  0.0425  296.353255    185.295558  102.437533       88.771778   75.227841   61.923447    54.173810   49.193989          NaN                                                              NaN
        NaN                   0.045  297.469245    186.430192  103.638409       90.000816   76.499329   63.264938    55.583248   50.668531          NaN                                                              NaN
        NaN                  0
Unnamed: 4  Unnamed: 5                      Unnamed: 6 Unnamed: 7 Unnamed: 8 Unnamed: 9  Unnamed: 10  Unnamed: 11 """

# Create a ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Score
scores = scorer.score(reference_summary, generated_summary)

# Print results
print("ROUGE Scores PEGASUS with SpaCy (Custom):")
for metric, score in scores.items():
    print(f"{metric}: Precision: {score.precision:.4f}, Recall: {score.recall:.4f}, F1: {score.fmeasure:.4f}")


ROUGE Scores PEGASUS with SpaCy (Custom):
rouge1: Precision: 0.0090, Recall: 0.1647, F1: 0.0170
rouge2: Precision: 0.0006, Recall: 0.0118, F1: 0.0012
rougeL: Precision: 0.0051, Recall: 0.0941, F1: 0.0097
